Author: Saman Paidar Nia

All resources are listed at the bottom of the page.

In [1]:
# Get important libraries for this class.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
import logging
import sys
#--------------------------------------------------------
from tensorflow.python.ops import control_flow_ops
from IPython.display import clear_output
from scipy.spatial.distance import squareform, pdist
from sklearn.preprocessing import normalize
from numpy import linalg as LA
from scipy.cluster.vq import kmeans, vq
from sklearn.metrics import normalized_mutual_info_score
from math import sqrt
#---------------------------------------------------------
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from optparse import OptionParser

In [2]:
class Similarity_Normalization:
    def normalization(self, X):
        return (X - min(X)) / (max(X) - min(X))

In [3]:
class Correlation_Similarity(Similarity_Normalization):
    def get_matrix(self, X):
        X = normalize(X, axis=0)
        X = pdist(X, 'correlation')
        X = squareform(X)
        S = np.sum(X, 0)
        D = np.diag(S)
        D = LA.matrix_power(D, -1)
        Y = np.dot(D, X)
        Y = np.apply_along_axis(self.normalization, 1, Y)
        np.fill_diagonal(Y, 0.)
        return Y

In [4]:
class Cosine_Similarity(Similarity_Normalization):
    def get_matrix(self, X):
        X = normalize(X, axis=0)
        X = pdist(X, 'cosine')
        X = squareform(X)
        S = np.sum(X, 0)
        D = np.diag(S)
        D = LA.matrix_power(D, -1)
        Y = np.dot(D, X)
        Y = np.apply_along_axis(self.normalization, 1, Y)
        np.fill_diagonal(Y, 0.)
        return Y

In [5]:
class Similarity_Dataset_Iterator():
    def __init__(self, data, labels, similarity):
        self.data = data
        self.labels = labels
        self.matrix = similarity.get_matrix(data)
        self.data_size = self.matrix.shape[0]
        self.current_index = 0
    def next_batch(self, num):
        data=self.matrix.transpose()
        labels=self.labels
        idx = np.arange(0 , len(data))
        np.random.shuffle(idx)
        idx = idx[:num]
        data_shuffle = [data[ i] for i in idx]
        labels_shuffle = [labels[ i] for i in idx]
        return data_shuffle, labels_shuffle
    def whole_dataset(self):
        return (self.matrix.transpose(), self.labels)

In [6]:
#  Using Scikit-Learn libraries to fetching the Newsgroups data set: http://scikit-learn.org
def read_NewsGroup_data(similarity):    
    logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
    op = OptionParser()
    op.add_option("--lsa", dest="n_components", type="int",
                  help="Preprocess documents with latent semantic analysis.")    
    op.add_option("--no-idf",action="store_false", dest="use_idf", default=True,
                  help="Disable Inverse Document Frequency feature weighting.")
    op.add_option("--use-hashing", action="store_true", default=False,
                  help="Use a hashing feature vectorizer")
    op.add_option("--n-features", type=int, default=10000,
                  help="Maximum number of features to extract from text.")    
    def is_interactive():
        return not hasattr(sys.modules['__main__'], '__file__')
    argv = [] if is_interactive() else sys.argv[1:]
    (opts, args) = op.parse_args(argv)
    if len(args) > 0:
        op.error("this script takes no arguments.")
        sys.exit(1)
    dataset = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)
    labels = dataset.target[:4000]
    true_k = np.unique(labels).shape[0]
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,min_df=2,
                                 stop_words='english',use_idf=opts.use_idf)
    X = vectorizer.fit_transform(dataset.data[:4000])
    if opts.n_components:
        svd = TruncatedSVD(opts.n_components)
        normalizer = Normalizer(copy=False)
        lsa = make_pipeline(svd, normalizer)
        X = lsa.fit_transform(X)
        explained_variance = svd.explained_variance_ratio_.sum()
    return Similarity_Dataset_Iterator(X.toarray(), labels, similarity)

In [7]:
# Call Cosine_Similarity as similarity dataset.
trainSet_cosine = read_NewsGroup_data(Cosine_Similarity())

In [8]:
# Call Correlation_Similarity as similarity dataset.
trainSet_correlation = read_NewsGroup_data(Correlation_Similarity())

In [9]:
n_input = trainSet_cosine.data_size #--------- Number of input data.
# Define the number of hidden layer. 
if n_input >= 2048:
    Nn = int(4096)
elif n_input >= 1024:
    Nn = int(2048)
elif n_input >= 512:
    Nn = int(1024)

In [10]:
n_hidden_1 = int(Nn/2) #-------------------- The autoencoder hidden layer 1.
n_hidden_2 = int(n_hidden_1/2) #------------ The autoencoder hidden layer 2.
n_hidden_3 = int(n_hidden_2/2) #------------ The autoencoder hidden layer 3.
n_code = str(int(n_hidden_3/2)) #----------- The number of output dimension value.

In [11]:
print('Layer 1: ', n_input)
print('Layer 2: ', n_hidden_1)
print('Layer 3: ', n_hidden_2)
print('Layer 4: ', n_hidden_3)
print('Layer 5: ', int(n_code))

Layer 1:  4000
Layer 2:  2048
Layer 3:  1024
Layer 4:  512
Layer 5:  256


In [12]:
def k_means_(X, n_cluster):
    kmeans_centroids,_ =  kmeans(X,n_cluster)
    kmeans_, _ = vq(X, kmeans_centroids)
    return kmeans_

In [13]:
def spectral_clustering(X, n_cluster):
    D_ = np.matrix(np.diag(np.sum(X, 0)))
    D_ = D_**(-1)
    L = np.dot(np.dot(D_, X), D_)
    eigenvectors = np.linalg.eig(L)[1]
    X = np.real(eigenvectors[:, 0: n_cluster])
    rows_norm = np.linalg.norm(X, axis=1, ord=0)
    X = (X.T / rows_norm).T
    return k_means_(X, n_cluster)

In [14]:
def encoder(x, n_code, phase_train):    
    with tf.variable_scope("encoder"):        
        with tf.variable_scope("hidden_1"):
            hidden_1 = layer(x, [n_input, n_hidden_1], [n_hidden_1], phase_train)
        with tf.variable_scope("hidden_2"):
            hidden_2 = layer(hidden_1, [n_hidden_1, n_hidden_2], [n_hidden_2], phase_train)
        with tf.variable_scope("hidden_3"):
            hidden_3 = layer(hidden_2, [n_hidden_2, n_hidden_3], [n_hidden_3], phase_train)        
        with tf.variable_scope("code"):
            code = layer(hidden_3, [n_hidden_3, n_code], [n_code], phase_train)
    return code

In [15]:
def decoder(code, n_code, phase_train):
    with tf.variable_scope("decoder"):
        with tf.variable_scope("hidden_1"):
            hidden_1 = layer(code, [n_code, n_hidden_3], [n_hidden_3], phase_train)
        with tf.variable_scope("hidden_2"):
            hidden_2 = layer(hidden_1, [n_hidden_3, n_hidden_2], [n_hidden_2], phase_train)
        with tf.variable_scope("hidden_3"):
            hidden_3 = layer(hidden_2, [n_hidden_2, n_hidden_1], [n_hidden_1], phase_train)              
        with tf.variable_scope("output"):
            output = layer(hidden_3, [n_hidden_1, n_input], [n_input], phase_train)
    return output

In [16]:
def layer_batch_norm(x, n_out, phase_train):
    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
    mean, var = control_flow_ops.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    return tf.reshape(normed, [-1, n_out])

In [17]:
def layer(input, weight_shape, bias_shape, phase_train):
    weight_init = tf.random_normal_initializer(stddev=(1.0 / weight_shape[0]) ** 0.5)
    bias_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    logits = tf.matmul(input, W) + b
    return tf.nn.sigmoid(layer_batch_norm(logits, weight_shape[1], phase_train))

In [18]:
def loss(output, x):
    with tf.variable_scope("training"):
        train_loss = tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        return train_loss

In [19]:
def training(cost, learning_rate, beta1, beta2, global_step):
    optimizer = tf.train.AdamOptimizer(learning_rate, beta1, beta2, epsilon=1e-08, use_locking=False, name='Adam')
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

In [20]:
# Parameters
n_layers = 5 #----------------------------- Number of Neural Networks Layers.
beta1 = 0.9 #------------------------------ The decay rate 1.  
beta2 = 0.999 #---------------------------- The decay rate 2.
learning_rate = (beta1/n_input) #---------- The learning rate.
stop_learning = 1.35 #--------------------- The stop learning point.
n_batch = math.ceil(sqrt(sqrt(n_input))) #- Number of selection data in per step.
n_backpro = math.ceil(n_input/n_batch) #--- Number of Backpro in per epoch.
n_cluster = 20 #--------------------------- Number of clusters.
n_diplay = 10 #---------------------------- Number of getting code and runnig the K-Means.

In [21]:
results_cor=[] #--------------------------- A list to keep all NMI scores.
loss_cost_cor=[] #------------------------- A list to keep all training evaluations.
steps_cor=[] #----------------------------- A list to keep all steps.

In [22]:
with tf.Graph().as_default():    
    with tf.variable_scope("autoencoder_model"):                
        x = tf.placeholder("float", [None, n_input])   
        phase_train = tf.placeholder(tf.bool)
        code = encoder(x, int(n_code), phase_train)
        output = decoder(code, int(n_code), phase_train)
        cost = loss(output, x)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_optimizer = training(cost, learning_rate, beta1, beta2, global_step)
        sess = tf.Session()
        init_op = tf.global_variables_initializer()
        sess.run(init_op)

In [23]:
# Training cycle
epoch = 0
first_pass = True
while first_pass or new_cost >= stop_learning:
    first_pass = False
    # Fit training with Backpropagation using batch data.
    for i in range(n_backpro):
        miniData, _ = trainSet_correlation.next_batch(n_batch)
        _, new_cost = sess.run([train_optimizer,cost], feed_dict={x: miniData,
                                                                  phase_train: True})       
    #------------------------- End of the Optimization ------------------------------
    epoch += 1
    # Save the results after per 10 epochs.    
    if epoch % n_diplay == 0 or new_cost <= stop_learning:
        # Getting embedded codes and running K-Means on them.
        ae_codes = sess.run(code, feed_dict={x: trainSet_correlation.whole_dataset()[0],
                                             phase_train: False})        
        idx = k_means_(ae_codes, n_cluster)
        ae_nmi_cor = normalized_mutual_info_score(trainSet_correlation.whole_dataset()[1], idx)
        results_cor.append(ae_nmi_cor)    
        steps_cor.append(epoch)
        loss_cost_cor.append(new_cost)    
        print("NMI Score for AE is: {:0.2f} and new cost is: {:0.2f} in {:d} step. "
              .format(ae_nmi_cor, new_cost, epoch))

NMI Score for AE is: 0.33 and new cost is: 257.88 in 10 step. 
NMI Score for AE is: 0.35 and new cost is: 62.17 in 20 step. 
NMI Score for AE is: 0.38 and new cost is: 14.14 in 30 step. 
NMI Score for AE is: 0.49 and new cost is: 3.80 in 40 step. 
NMI Score for AE is: 0.59 and new cost is: 1.90 in 50 step. 
NMI Score for AE is: 0.59 and new cost is: 1.54 in 60 step. 
NMI Score for AE is: 0.60 and new cost is: 1.44 in 70 step. 
NMI Score for AE is: 0.61 and new cost is: 1.33 in 75 step. 


In [24]:
results_cos=[] #--------------------------- A list to keep all NMI scores.
loss_cost_cos=[] #------------------------- A list to keep all training evaluations.
steps_cos=[] #----------------------------- A list to keep all steps.

In [25]:
with tf.Graph().as_default():    
    with tf.variable_scope("autoencoder_model"):                
        x = tf.placeholder("float", [None, n_input])   
        phase_train = tf.placeholder(tf.bool)
        code = encoder(x, int(n_code), phase_train)
        output = decoder(code, int(n_code), phase_train)
        cost = loss(output, x)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_optimizer = training(cost, learning_rate, beta1, beta2, global_step)
        sess = tf.Session()
        init_op = tf.global_variables_initializer()
        sess.run(init_op)

In [26]:
# Training cycle
epoch = 0
first_pass = True
while first_pass or new_cost >= stop_learning:
    first_pass = False
    # Fit training with Backpropagation using batch data.
    for i in range(n_backpro):
        miniData, _ = trainSet_cosine.next_batch(n_batch)
        _, new_cost = sess.run([train_optimizer,cost], feed_dict={x: miniData,
                                                                  phase_train: True})       
    #------------------------- End of the Optimization ------------------------------
    epoch += 1
    # Save the results after per 10 epochs.    
    if epoch % n_diplay == 0 or new_cost <= stop_learning:
        # Getting embedded codes and running K-Means on them.
        ae_codes = sess.run(code, feed_dict={x: trainSet_cosine.whole_dataset()[0],
                                             phase_train: False})        
        idx = k_means_(ae_codes, n_cluster)
        ae_nmi_cos = normalized_mutual_info_score(trainSet_cosine.whole_dataset()[1], idx)
        results_cos.append(ae_nmi_cos)    
        steps_cos.append(epoch)
        loss_cost_cos.append(new_cost)    
        print("NMI Score for AE is: {:0.2f} and new cost is: {:0.2f} in {:d} step. "
              .format(ae_nmi_cos, new_cost, epoch))

NMI Score for AE is: 0.32 and new cost is: 268.83 in 10 step. 
NMI Score for AE is: 0.33 and new cost is: 66.90 in 20 step. 
NMI Score for AE is: 0.33 and new cost is: 16.52 in 30 step. 
NMI Score for AE is: 0.48 and new cost is: 4.46 in 40 step. 
NMI Score for AE is: 0.54 and new cost is: 2.00 in 50 step. 
NMI Score for AE is: 0.53 and new cost is: 1.53 in 60 step. 
NMI Score for AE is: 0.55 and new cost is: 1.46 in 70 step. 
NMI Score for AE is: 0.55 and new cost is: 1.59 in 80 step. 
NMI Score for AE is: 0.57 and new cost is: 1.43 in 90 step. 
NMI Score for AE is: 0.58 and new cost is: 1.66 in 100 step. 


KeyboardInterrupt: 

In [ ]:
spec_cluster_cos = spectral_clustering(trainSet_cosine.whole_dataset()[0], n_cluster)
spec_nmi_cos = normalized_mutual_info_score(trainSet_cosine.whole_dataset()[1], spec_cluster_cos)
#-----------------------------------------------------------------------------------------------------------
k_means_indx_cos = k_means_(trainSet_cosine.whole_dataset()[0], n_cluster)
k_means_nmi_cos = (normalized_mutual_info_score(trainSet_cosine.whole_dataset()[1], k_means_indx_cos))
#-----------------------------------------------------------------------------------------------------------
spec_cluster_cor = spectral_clustering(trainSet_correlation.whole_dataset()[0], n_cluster)
spec_nmi_cor = normalized_mutual_info_score(trainSet_correlation.whole_dataset()[1], spec_cluster_cor)
#-----------------------------------------------------------------------------------------------------------
k_means_indx_cor = k_means_(trainSet_correlation.whole_dataset()[0], n_cluster)
k_means_nmi_cor = (normalized_mutual_info_score(trainSet_correlation.whole_dataset()[1], k_means_indx_cor))

In [ ]:
warnings.filterwarnings('ignore')
plt.figure(figsize=(18, 6))
plt.subplot(1,2,1)
plt.errorbar(steps_cor, loss_cost_cor, label='Cost Trianing for Correlation Distance ', color='#000080', marker='o')
plt.errorbar(steps_cos, loss_cost_cos, label='Cost Trianing for Cosine Distance ', color='#E3CF57', marker='o')
plt.xlabel('Number of Epochs.')
plt.ylabel('Cost')
plt.grid()
plt.title('Cost Function Trianing')
plt.legend(loc='best')
plt.subplot(1,2,2)
plt.errorbar(steps_cor, results_cor, label='AE Normalized Correlation Distance ', color='#000080', marker='o')
plt.errorbar(steps_cos, results_cos, label='AE Normalized Cosine Distance ', color='#E3CF57', marker='o')
plt.xlabel('Number of Epochs.')
plt.ylabel('NMI')
plt.grid()
plt.title(('NMI of AE Correlation is {:0.2f} and AE Cosine is {:0.2f}').format(ae_nmi_cor*100, ae_nmi_cos*100))
plt.legend(loc='best')
plt.show()

In [ ]:
print("KMeans on Cosine: ---------------------------- {:0.2f}".format(k_means_nmi_cos*100))
print("KMeans on Correlation: ----------------------- {:0.2f}".format(k_means_nmi_cor*100))
print("Spectral Clustering on Cosine: --------------- {:0.2f}".format(spec_nmi_cos*100))
print("Spectral Clustering on Correlation: ---------- {:0.2f}".format(spec_nmi_cor*100))
print("Autoencoder Clustering on Cosine: ------------ {:0.2f}".format(ae_nmi_cos*100))
print("Autoencoder Clustering on Correlation: ------- {:0.2f}".format(ae_nmi_cor*100))

____________________________________________________________________________________________________________________________

____________________________________________________________________________________________________________________________

Refrences:

SciPy (the library) Jones E, Oliphant E, Peterson P, et al. SciPy: Open Source Scientific Tools for Python, 2001-, http://www.scipy.org/ [Online; accessed 2018-02-20]. Here’s an example of a BibTeX entry:

@Misc{, author = {Eric Jones and Travis Oliphant and Pearu Peterson and others}, title = {{SciPy}: Open source scientific tools for {Python}}, year = {2001--}, url = "http://www.scipy.org/", note = {[Online; accessed

NumPy & SciPy:

Stéfan van der Walt, S. Chris Colbert and Gaël Varoquaux. The NumPy Array: A Structure for Efficient Numerical Computation, Computing in Science & Engineering, 13, 22-30 (2011), DOI:10.1109/MCSE.2011.37 (publisher link)

IPython:

Fernando Pérez and Brian E. Granger. IPython: A System for Interactive Scientific Computing, Computing in Science & Engineering, 9, 21-29 (2007), DOI:10.1109/MCSE.2007.53 (publisher link)

Matplotlib:

John D. Hunter. Matplotlib: A 2D Graphics Environment, Computing in Science & Engineering, 9, 90-95 (2007), DOI:10.1109/MCSE.2007.55 (publisher link)

Scikit-learn:

Fabian Pedregosa, Gaël Varoquaux, Alexandre Gramfort, Vincent Michel, Bertrand Thirion, Olivier Grisel, Mathieu Blondel, Peter Prettenhofer, Ron Weiss, Vincent Dubourg, Jake Vanderplas, Alexandre Passos, David Cournapeau, Matthieu Brucher, Matthieu Perrot, Édouard Duchesnay. Scikit-learn: Machine Learning in Python, Journal of Machine Learning Research, 12, 2825-2830 (2011) (publisher link)

TensorFlow:

@misc{tensorflow2015-whitepaper, title={ {TensorFlow}: Large-Scale Machine Learning on Heterogeneous Systems}, url={https://www.tensorflow.org/}, note={Software available from tensorflow.org},

Jupyter Notebooks:

@conference{Kluyver:2016aa, Author = {Thomas Kluyver and Benjamin Ragan-Kelley and Fernando P{\'e}rez and Brian Granger and Matthias Bussonnier and Jonathan Frederic and Kyle Kelley and Jessica Hamrick and Jason Grout and Sylvain Corlay and Paul Ivanov and Dami{\'a}n Avila and Safia Abdalla and Carol Willing}, Booktitle = {Positioning and Power in Academic Publishing: Players, Agents and Agendas}, Editor = {F. Loizides and B. Schmidt}, Organization = {IOS Press}, Pages = {87 - 90}, Title = {Jupyter Notebooks -- a publishing format for reproducible computational workflows}, Year = {2016}}